In [43]:
# Obtiene los datos de precipitación par la Cuenca del Valle
# de Méxco a partir de CHIRPS y los promedia espacialmente.

import os

import rioxarray

import pandas as pd
import numpy as np

import geopandas as gpd
import xarray as xr

# Datos
path_d = "../results/onset/"
path_r = "../results/onset/graficas/"

# Variable, nivel de presión y región.
vars = [ "u", "v", "gp", "sst", "olr", "sp" ]
levels = [ "925", "200" ]
region = [ "mexico" ]

# Cantidad de días promediados.
d = 20

# Fechas de onset por precipitación
date_pre = pd.read_csv(path_d + "onset_CHIRPS.csv", index_col = "Año" )
date_pre.index = pd.RangeIndex(date_pre.index[0], date_pre.index[-1] + 1, 
    name = "Año" )
date_pre.iloc[:, 0] = pd.to_datetime( date_pre.iloc[:, 0] )
date_pre.iloc[:, 1] = pd.to_datetime( date_pre.iloc[:, 1] )

# Se convierten las fechas a días julianos.
date_pre["Cuenca - Onset"] = date_pre["Cuenca - Onset"].apply(
    lambda x: pd.to_datetime(x) )
date_pre["Cuenca - Onset"] = date_pre["Cuenca - Onset"].apply(
    lambda x: x.dayofyear )

In [44]:
def onset( df, vars = [-12.5, 7] , m_i = 5 ):

    umbral = vars[0]
    n = int( vars[1] )

    c = "Cuenca"

    # Nombre de las columnas dummy.
    proc = [c]
    for i in range( 0, n ): proc.append(c + "_" + str(i))

    # Traslapa los últimos 3 días 
    # y da un valor de 1 a cada día 
    # que cumpla con el umbral.
    # Umbral positivo.
    if vars[0] >= 0:
        for i, p in enumerate(proc[1:]):
            df[p] = df[proc[0]].shift(i)
            df[p].clip( upper = umbral, inplace = True )
            df[p] = df[p].where( df[p] == umbral, 0 )
            df[p] = df[p].where( df[p] < umbral, 1 )
    # Umbral negativo
    if vars[0] < 0:
        for i, p in enumerate(proc[1:]):
            df[p] = df[proc[0]].shift(i)
            df[p].clip( lower = umbral, inplace = True )
            df[p] = df[p].where( df[p] == umbral, 0 )
            df[p] = df[p].where( df[p] > umbral, 1 )

    # El índice es uno si los n
    # cumplen con el umbral.
    ind = "Index_" + c
    df[ind] = 0
    for i in range( n ):
        df[ind] += df[proc[i + 1]]
    df[ind] = df[ind].where( df[ind] == n, 0 )
    df[ind] = df[ind].where( df[ind] < n, 1 )
    df[ind] = df[ind].where( df.index.month.isin( range(m_i, 12) ), 0 )
    df.drop(proc[1:], axis = 1, inplace = True)

    # Crea una tabla con los valores anuales.
    date = date_pre["Cuenca - Onset"].copy()
    years = range(1981, 2021)

    for j in years:
        # Solo utilizamos valores entre mayo y noviembre.
        df_year = df[ (df.index.year == j)
            & df.index.month.isin(range(m_i, 12)) ]

        # Fechas.
        date[j] = df_year.iloc[:, 1].idxmax()

    # Damos formato a las columnas y al índice.
    date.index = years
    # Recorremos al inicio de la racha de días de precipitación.
    date = date - np.timedelta64(n, 'D')

    # Correlación entre índice de precipitación y de viento.

    # Se convierten las fechas a días julianos.
    date = date.apply( lambda x: pd.to_datetime(x) )
    date = date.apply( lambda x: x.dayofyear )

    # Correlaciones de las series.
    corr = date.corr( date_pre["Cuenca - Onset"] )

    # Liberamos memoria.
    del date

    return corr

def ind_max( ds, zone = (-80, -70, 12.5, 17.5), n = (1, 20), num = 10 ):
    # Precipitación diaria promedio en toda la cuenca.
    data = ( ds.sel( longitude = slice( zone[0], zone[1] ), 
        latitude = slice( zone[3], zone[2] ) )
        .mean(dim = ["latitude", "longitude"]).to_dataframe() )
    data.columns = ["Cuenca"]

    u = [ np.fix( data.min()[0] * 2 ) / 2,
        np.fix( data.max()[0] * 2 ) / 2, num ]

    c = np.stack( np.meshgrid( np.linspace( *u ), np.linspace( *n ) ),
        axis = 2 )
    res = np.ndarray(shape = c.shape[0:2])

    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            res[i, j] = onset( data, c[i, j] )

    return ( res.max(), c[ np.argmax(res) // res.shape[1],
        np.argmax(res) % res.shape[1] ], u )

def eval(v = 0, n = 0, step = 20, num = 5):
    # Variable a utilizar.
    if v > 2: lev = ""
    else: lev = "_" + levels[n]
    fname = "onset_" + vars[v] + lev + "_mean_" + str(d) + "_dias.nc"
    ds = xr.open_dataset( path_d + fname)

    lon = np.arange(-120, -50 + step, step)
    lat = np.arange(0 , 40 + step, step)

    lon = np.stack( [ lon[0:-1], lon[1:] ], axis = 1)
    lat = np.stack( [ lat[0:-1], lat[1:] ], axis = 1)

    a = np.repeat(lon, lat.shape[0], axis = 0)
    b = np.tile(lat, [lon.shape[0], 1])

    bounds = np.concatenate([a, b], axis = 1)

    x = []
    y = np.ndarray([0])

    for i in bounds:
        z = ind_max( ds, zone = i, num = num )
        x.append( z )
        y = np.append( y, [z[0]], axis = 0 )

    print( bounds[ np.argmax(y) ] )
    print( x[ np.argmax(y) ] )

In [45]:
eval(v = 0, n = 0)

[-100  -80    0   20]
(0.43242800353993055, array([ 1.25      , 14.18367347]), [-5.5, 3.5, 5])


In [46]:
eval(v = 0, n = 1)

[-100  -80    0   20]
(0.37203417019338453, array([ 5.5      , 19.2244898]), [-13.0, 24.0, 5])


In [50]:
eval(v = 1, n = 0)

[-60 -40  20  40]
(0.42796311598174325, array([2.875, 1.   ]), [-5.0, 5.5, 5])


In [47]:
eval(v = 1, n = 1)

[-100  -80    0   20]
(0.3842367367326, array([-2., 20.]), [-7.5, 14.5, 5])


In [54]:
eval(v = 2, n = 0)

[-80 -60   0  20]
(0.3902023350969838, array([302.5,  20. ]), [298.5, 302.5, 5])


In [55]:
eval(v = 2, n = 1)

KeyboardInterrupt: 

In [51]:
eval(v = 3)

[-80 -60   0  20]
(0.3902023350969838, array([302.5,  20. ]), [298.5, 302.5, 5])


In [52]:
eval(v = 4)

[-100  -80   20   40]
(0.3783809576930082, array([-264.       ,   19.2244898]), [-277.0, -225.0, 5])


In [53]:
eval(v = 5)

[-100  -80   20   40]
(0.41369218439777344, array([9.98155000e+04, 7.20408163e+00]), [99051.0, 100580.0, 5])
